## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [107]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.datasets import make_moons
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error, accuracy_score

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

Notes:

Learning rate is: # how much are we going to change the parameter value. at a given step along optimization, we use gradient that gives biggest decrease in loss function (thing we are trying to minimize)

Uniform is a square distribution, it is saying start at the first, the second value is the length of the range. Values from 0.01 - 0.3. (0.3 is the length of the range) 

In [29]:
# ---- Define X & y variables ----

y = model_data['Angaus']
X = model_data.drop('Angaus', axis = 1)

In [58]:
# ---- Encode X data ----

# Initialize the encoder
le = LabelEncoder()

# Encode the "Method" column
X["Method"] = le.fit_transform(X["Method"])

# ---- Split X into test and train for early stopping ----

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [59]:
# ---- Define and fit baseline model ----

# Define XGBoost model
model = xgb.XGBClassifier(n_estimators = 1000,
                          early_stopping_rounds = 50,
                          learning_rate = 0.1,
                          eval_metric = "logloss",
                          random_state = 808)

# Fit the model
model.fit(X_train,
          y_train,
         eval_set=[(X_test, y_test)])

[0]	validation_0-logloss:0.64173
[1]	validation_0-logloss:0.59874
[2]	validation_0-logloss:0.56352
[3]	validation_0-logloss:0.53716
[4]	validation_0-logloss:0.51702
[5]	validation_0-logloss:0.50016
[6]	validation_0-logloss:0.48135
[7]	validation_0-logloss:0.46672
[8]	validation_0-logloss:0.45634
[9]	validation_0-logloss:0.44405
[10]	validation_0-logloss:0.43728
[11]	validation_0-logloss:0.42904
[12]	validation_0-logloss:0.41955
[13]	validation_0-logloss:0.41371
[14]	validation_0-logloss:0.40708
[15]	validation_0-logloss:0.40332
[16]	validation_0-logloss:0.39853
[17]	validation_0-logloss:0.39618
[18]	validation_0-logloss:0.39130
[19]	validation_0-logloss:0.38952
[20]	validation_0-logloss:0.38569
[21]	validation_0-logloss:0.38372
[22]	validation_0-logloss:0.37811
[23]	validation_0-logloss:0.37809
[24]	validation_0-logloss:0.37631
[25]	validation_0-logloss:0.37448
[26]	validation_0-logloss:0.37215
[27]	validation_0-logloss:0.37295
[28]	validation_0-logloss:0.37361
[29]	validation_0-loglos

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              predictor='auto', random_state=808, reg_alpha=0, reg_lambda=1, ...)

In [69]:
print(f"Best number of trees for the baseline model: {model.best_ntree_limit}")
best_trees = model.best_ntree_limit

Best number of trees for the baseline model: 35


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [47]:
# Define hyperparameter distributions
param_dist = {
    "learning_rate": uniform(0.01, 0.3),
}

# Set up RandomizedSearchCV (ranges for hyperparameters, randomly selects values in tuning)
random_search = RandomizedSearchCV(
    model,
    param_dist,
    n_iter = 20,
    random_state = 808)

# Fit the model
random_search.fit(X_train, y_train,
         eval_set=[(X_test, y_test)]) # do I need this

# do I need to add other options
# eval_set=[(X_test, y_test)], eval_metric="logloss", early_stopping_rounds=50

[0]	validation_0-logloss:0.56935
[1]	validation_0-logloss:0.50283
[2]	validation_0-logloss:0.46452
[3]	validation_0-logloss:0.43969
[4]	validation_0-logloss:0.42207
[5]	validation_0-logloss:0.41589
[6]	validation_0-logloss:0.40949
[7]	validation_0-logloss:0.40428
[8]	validation_0-logloss:0.40027
[9]	validation_0-logloss:0.39791
[10]	validation_0-logloss:0.39161
[11]	validation_0-logloss:0.38388
[12]	validation_0-logloss:0.39086
[13]	validation_0-logloss:0.39352
[14]	validation_0-logloss:0.38521
[15]	validation_0-logloss:0.37883
[16]	validation_0-logloss:0.38336
[17]	validation_0-logloss:0.37931
[18]	validation_0-logloss:0.38911
[19]	validation_0-logloss:0.39051
[20]	validation_0-logloss:0.39466
[21]	validation_0-logloss:0.39574
[22]	validation_0-logloss:0.39483
[23]	validation_0-logloss:0.39364
[24]	validation_0-logloss:0.39584
[25]	validation_0-logloss:0.39609
[26]	validation_0-logloss:0.40101
[27]	validation_0-logloss:0.40180
[28]	validation_0-logloss:0.40386
[29]	validation_0-loglos

RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss', gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0.1, max_bin=None,
                                           max...
                                           max_delta_step=None, max_depth=None,
                                  

In [71]:
print(f"Best learning rate: {random_search.best_params_['learning_rate']}")

best_learning_rate = random_search.best_params_['learning_rate']

Best learning rate: 0.19644487306375513


KeyError: 'n_estimators'

### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 

In [85]:
# ---- Tune tree specific paraemters  ----

# Define XGBoost model
model = xgb.XGBClassifier(
    n_estimators = best_trees,
    learning_rate  = best_learning_rate,
 #   eval_metric = "logloss", # should I keep these
    random_state = 808
)

# Define parameter dictionary
param_dist = {
    'max_depth': randint(3, 11),
    'min_child_weight': randint(1, 11),
    'gamma': uniform(0.05, 0.05),
}

# Find best combination of parameteres
random_search = RandomizedSearchCV(
    model,
    param_dist,
    n_iter = 20, # should I keep this
    random_state=808,
    cv = 5
)


# Fit the model to find the best parameters
random_search.fit(X_train, y_train,
                 eval_set=[(X_test, y_test)]) # do I need these



[0]	validation_0-logloss:0.60480
[1]	validation_0-logloss:0.53759
[2]	validation_0-logloss:0.49564
[3]	validation_0-logloss:0.46215
[4]	validation_0-logloss:0.43767
[5]	validation_0-logloss:0.42129
[6]	validation_0-logloss:0.40793
[7]	validation_0-logloss:0.39817
[8]	validation_0-logloss:0.39229
[9]	validation_0-logloss:0.38886
[10]	validation_0-logloss:0.38463
[11]	validation_0-logloss:0.37783
[12]	validation_0-logloss:0.37934
[13]	validation_0-logloss:0.37696
[14]	validation_0-logloss:0.37675
[15]	validation_0-logloss:0.37783
[16]	validation_0-logloss:0.37475
[17]	validation_0-logloss:0.37399
[18]	validation_0-logloss:0.37130
[19]	validation_0-logloss:0.37189
[20]	validation_0-logloss:0.37383
[21]	validation_0-logloss:0.37263
[22]	validation_0-logloss:0.37362
[23]	validation_0-logloss:0.36958
[24]	validation_0-logloss:0.36756
[25]	validation_0-logloss:0.36654
[26]	validation_0-logloss:0.36332
[27]	validation_0-logloss:0.36626
[28]	validation_0-logloss:0.36456
[29]	validation_0-loglos

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0.1964448730637551...
                                           predictor=None, random_state=808,
                                           reg_alpha=None, reg

In [91]:
print("Best stochastic params:", random_search.best_params_)

best_stochastic_params = random_search.best_params_

Best stochastic params: {'colsample_bytree': 0.8506612141420485, 'subsample': 0.7565024905664723}


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [89]:
# Define dictionary
param_dist = {
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5)
}

# Find best combination of parameters
random_search = RandomizedSearchCV(
    model,
    param_dist,
    random_state = 808)

random_search.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)])

random_search.best_params_

[0]	validation_0-logloss:0.61050
[1]	validation_0-logloss:0.54988
[2]	validation_0-logloss:0.49960
[3]	validation_0-logloss:0.46523
[4]	validation_0-logloss:0.44444
[5]	validation_0-logloss:0.42518
[6]	validation_0-logloss:0.41193
[7]	validation_0-logloss:0.39856
[8]	validation_0-logloss:0.38934
[9]	validation_0-logloss:0.38873
[10]	validation_0-logloss:0.38592
[11]	validation_0-logloss:0.38837
[12]	validation_0-logloss:0.38472
[13]	validation_0-logloss:0.38527
[14]	validation_0-logloss:0.38606
[15]	validation_0-logloss:0.39047
[16]	validation_0-logloss:0.39151
[17]	validation_0-logloss:0.38745
[18]	validation_0-logloss:0.38655
[19]	validation_0-logloss:0.38407
[20]	validation_0-logloss:0.39123
[21]	validation_0-logloss:0.38631
[22]	validation_0-logloss:0.39166
[23]	validation_0-logloss:0.39235
[24]	validation_0-logloss:0.39548
[25]	validation_0-logloss:0.39468
[26]	validation_0-logloss:0.39185
[27]	validation_0-logloss:0.39024
[28]	validation_0-logloss:0.38947
[29]	validation_0-loglos

{'colsample_bytree': 0.8506612141420485, 'subsample': 0.7565024905664723}

In [92]:
print("Best stochastic params:", random_search.best_params_)

best_stochastic_params = random_search.best_params_

Best stochastic params: {'colsample_bytree': 0.8506612141420485, 'subsample': 0.7565024905664723}


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [ ]:
# ---- Define X & y variables ----

y_test = eval_data['Angaus']
X_test = eval_data.drop('Angaus', axis = 1)


In [108]:
# ---- Encode X data ----

# Initialize the encoder
le = LabelEncoder()

# Encode the "Method" column
X_test["Method"] = le.fit_transform(X_test["Method"])


# ---- Define the model ----


# Define XGBoost model
final_model = xgb.XGBClassifier(n_estimators = best_trees, # updated
                  #        early_stopping_rounds = 50,
                          learning_rate = best_learning_rate, #updated
                          eval_metric = "logloss", # updated
                          random_state = 808, # updated
                          colsample_bytree = 0.8506612141420485, # updated
                          subsample = 0.7565024905664723) # updated

# Fit the model
final_model.fit(X, y)

# ---- Evaluate the model ----

#final_model.evaluate(X_test, y_test)

logloss = log_loss(y_test, final_model.predict_proba(X_test)) # should I use log_loss here

y_pred = final_model.predict(X_test)
fm_accuracy = accuracy_score(y_test, y_pred)


print(f"Log Loss: {logloss}")
print(fm_accuracy)

Log Loss: 0.11668260580976493
0.99


### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?



- interpretability
- difficulty in creating it
- computational power


